In [1]:
%matplotlib qt

import neuroseries as nts

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from matplotlib import cm

import scipy.stats
import scipy.ndimage
import scipy.signal

import os

import csv

import cv2
import datetime

import bk.load 
import bk.plot
import bk.compute

import sklearn.manifold as skm

In [137]:
def pourcent_freezing(interval, freezing_intervals): 
    freezing_intervals_in_interval = freezing_intervals.intersect(interval)
    total_time_freezing = np.sum(freezing_intervals_in_interval.end-freezing_intervals_in_interval.start)
    total_time = interval.end-interval.start
    pourcentage = (total_time_freezing/total_time)*100
    return pourcentage

def all_pourcentages(intervals, freezing_intervals):
    pourcentages = []
    for i in range(len(intervals)):
        interval = intervals.loc[i]
        pourcentages.append(pourcent_freezing(interval, freezing_intervals))
    return pourcentages

def GetStartExp(Exp,Fs = 20000):
    
    if isinstance(Exp[0],(np.bool_,bool)):
        Exp = list(map(int,Exp))
    
    diff_Exp = np.diff(Exp)
    
    t_start = np.where(diff_Exp == 1)[0]    
    return t_start/Fs

In [55]:
import pandas as pd 

def DLC_posRatChelou(filtered = True,start = 0):
    """
    Load position from DLC files (*.h5) and returns it as a nts.TsdFrame
    """
    
    files = os.listdir()
    for f in files:
        if filtered and f.endswith('filtered.h5'): 
            filename = f
            break
        if not filtered and not f.endswith('filtered.h5') and f.endswith('.h5'):
            filename = f
            break
            
    data = pd.read_hdf(filename)
    data = data[data.keys()[0][0]]
    
    TTL = bk.load.digitalin('digitalin.dat')[0,:]
    TTL = TTL[start:]
    tf = bk.compute.TTL_to_times(TTL)
    
    if len(tf)>len(data):
        tf = np.delete(tf,-1)
    
    data.index = tf * 1_000_000
    pos = nts.TsdFrame(data)
    return pos

## Analysis for classical fear conditionning 

In [12]:
context_sess = ['Z://classical_fear_conditionning\Rat09-20201118\Rat09_201118_180813',
                'Z://classical_fear_conditionning\Rat10-20201118\Rat10_201118_182234', 
                'Z://classical_fear_conditionning\Rat11-20201117\Rat11_201117_172705',
                'Z://classical_fear_conditionning\Rat12-20201117\Rat12_201117_171302']
fear_sess = ['Z://classical_fear_conditionning\Rat09-20201118\Rat09_201118_110003',
             'Z://classical_fear_conditionning\Rat10-20201118\Rat10_201118_111531',
             'Z://classical_fear_conditionning\Rat11-20201117\Rat11_201117_112130',
             'Z://classical_fear_conditionning\Rat12-20201117\Rat12_201117_110146']

tone_sess = ['Z://classical_fear_conditionning\Rat09-20201118\Rat09_201118_170529', 
        'Z://classical_fear_conditionning\Rat10-20201118\Rat10_201118_172028', 
        'Z://classical_fear_conditionning\Rat11-20201117\Rat11_201117_183012',
        'Z://classical_fear_conditionning\Rat12-20201117\Rat12_201117_181452']


intervals = np.linspace(0,570,20)
starts = intervals[0:-1]
ends = intervals[1:]
intervals = nts.IntervalSet(starts,ends,time_units='s')

digitalin_tone = [3,3,3,3,2,2]
digitalin_shock = [2,2,2,2,1,1]

In [16]:
ptotal = []
plt.figure()
plt.subplot(3,1,1)

for fichier in fear_sess:
    os.chdir(fichier)
    exp, shock, tone = bk.compute.intervals_exp()
    t_s = exp.start[0]
    pos = bk.load.DLC_pos()
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s 
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)
tone = tone - t_s
shock = shock - t_s
bk.plot.intervals(shock,'green',alpha = 0.5)    
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat09','Rat10','Rat11','Rat12','Mean','Shock','Tone'])
plt.title('Pourcent freezing during fear conditionning')

Text(0.5, 1.0, 'Pourcent freezing during fear conditionning')

In [17]:
ptotal = []
plt.subplot(3,1,2)

for fichier in context_sess:
    os.chdir(fichier)
    exp, shock, tone = bk.compute.intervals_exp()
    t_s = exp.start[0]
    pos = bk.load.DLC_pos()
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s 
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)  
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat09','Rat10','Rat11','Rat12', 'Mean'])
plt.title('Pourcent freezing during context recall')

Text(0.5, 1.0, 'Pourcent freezing during context recall')

In [18]:
ptotal = []
plt.subplot(3,1,3)

for fichier in tone_sess:
    os.chdir(fichier)
    exp, shock, tone = bk.compute.intervals_exp()
    t_s = exp.start[0]
    pos = bk.load.DLC_pos()
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s 
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)   
tone = tone - t_s
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat09','Rat10','Rat11','Rat12','Mean','Tone'])
plt.title('Pourcent freezing during tone recall')

Y:\Python\Maelle\Amy-Hpc-sleep-dynamics-python\bk\compute.py:161: RuntimeWarning: overflow encountered in square
  v = np.sqrt(x_speed**2 + y_speed**2)


Text(0.5, 1.0, 'Pourcent freezing during tone recall')

In [4]:
for dirs,f,fe in os.walk('Y://classical_fear_conditionning/'):
    os.chdir(dirs)
    try: bk.load.DLC_pos(save = True)
    except: print(dirs)
    try: bk.compute.intervals_exp(save = True)
    except: print(dirs) 

In [6]:
t_start_exp_tone = [49.9391, 12.74455, 30.36235, 29.62595, 58.95, 17.8116]

ptotal = []
plt.figure()

for fichier,t_s,d_t in zip(tone_sess,t_start_exp_tone,digitalin_tone):
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    pos.index = pos.index
    tone = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_t,:])- t_s * 1_000_000
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.5)
    
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-r')
plt.legend(['Rat09', 'Rat10', 'Rat11', 'Rat12', 'Rat01', 'Rat02','Mean','Tone'])

Z:\Python\Maelle\Amy-Hpc-sleep-dynamics-python\bk\compute.py:161: RuntimeWarning: overflow encountered in square
  v = np.sqrt(x_speed**2 + y_speed**2)


In [60]:
t_start_exp_fear = [37.7209,29.46495,34.7118,25.34385,45.9524,55.00735]

ptotal = []
plt.figure()

for fichier,t_s,d_s in zip(fear_sess,t_start_exp_fear, digitalin_shock):
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    pos.index = pos.index
    tone = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_s+1,:])- t_s * 1_000_000
    shock = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_s,:])- t_s * 1_000_000
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.5)
bk.plot.intervals(shock,'green',alpha = 0.5)    
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-r')
plt.legend(['Rat09', 'Rat10', 'Rat11', 'Rat12', 'Rat01', 'Rat02','Mean','Shock','Tone'])

In [62]:
t_start_exp_context = [34,25,36,26,38,58]

ptotal = []
plt.figure()

for fichier,t_s in zip(context_sess,t_start_exp_context):
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    pos.index = pos.index
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.5)
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-r')
plt.legend(['Rat09', 'Rat10', 'Rat11', 'Rat12', 'Rat01', 'Rat02','Mean'])

In [60]:
# t_start_exp_fear = [37.7209,29.46495,34.7118,25.34385,45.9524,55.00735]

# ptotal = []

# for subsession,t_s,d_s in zip(fear_sess,t_start_exp_fear, digitalin_shock):
#     os.chdir(subsession)
#     pos = bk.load.DLC_pos()
#     tone = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_s+1,:])- t_s * 1_000_000 
#     shock = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_s,:])- t_s * 1_000_000
#     speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
#     freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
#     pourcents = all_pourcentages(intervals, freeze)
#     ptotal.append(pourcents)
# sem = scipy.stats.sem(ptotal, axis=0)
# print(sem)

[ 3.18776974  1.48582465  3.23840677  0.64798611 11.55357156 13.12103988
 12.30773501 11.73506083 10.50992606 14.03412223 16.89255854 15.27001727
 16.21988447  7.17805611  7.52125021 17.89187624 18.49912287 14.66015871
 13.17433428]


In [15]:
# t_start_exp_fear = [37.7209,29.46495,34.7118,25.34385,45.9524,55.00735]

# ptotal = []
# plt.figure()

# for fichier,t_s,d_s in zip(fear_sess,t_start_exp_fear, digitalin_shock):
#     os.chdir(fichier)
#     pos = bk.load.DLC_pos()
#     pos.index = pos.index
#     tone = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_s+1,:])- t_s * 1_000_000
#     shock = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_s,:])- t_s * 1_000_000
#     speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
#     freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
#     pourcents = all_pourcentages(intervals, freeze)
#     ptotal.append(pourcents)
# #     bk.plot.intervals(freeze,col = 'b')
#     plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.5)
# sem = scipy.stats.sem(ptotal, axis=0)
# bk.plot.intervals(shock,'green',alpha = 0.5)    
# bk.plot.intervals(tone,'orange', alpha=0.5)    
# plt.errorbar(np.mean([starts,ends],0),np.mean(ptotal,0),sem,'*-r')
# plt.legend(['Rat09', 'Rat10', 'Rat11', 'Rat12', 'Rat01', 'Rat02','Mean','Shock','Tone'])

ValueError: The lengths of the data (19) and the error 3 do not match

In [61]:
for fichier in context_sess:
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    print(pos.as_units('s').index[-1]-575)
#     f_tone = GetStartExp(bk.load.digitalin('digitalin.dat')[1,:])
#     print(f_tone[f_tone>0])
    
    


34.458774999999946
[  3.0225    3.1475    3.2725  203.08695 353.30415 503.51535]
25.68372499999998
[  3.0225    3.1475    3.2725  203.08695 353.30415 503.51535]
36.234050000000025
[  3.0225    3.1475    3.2725  203.08695 353.30415 503.51535]
26.03037500000005
[  3.0225    3.1475    3.2725  203.08695 353.30415 503.51535]
38.39945
[  3.0225    3.1475    3.2725  203.08695 353.30415 503.51535]
58.11689999999999
[  3.0225    3.1475    3.2725  203.08695 353.30415 503.51535]


## Analysis for PTSD fear conditionning 

In [10]:
neutral_sess = ['Y://fear_conditionning_ptsd\Rat13\Rat13-20201202\Rat13_201202_183404',
               'Y://fear_conditionning_ptsd\Rat14\Rat14-20201203\Rat14_201203_165203',
               'Y://fear_conditionning_ptsd\Rat15\Rat15-20201130\Rat15_201130_164002',
               'Y://fear_conditionning_ptsd\Rat16\Rat16-20201201\Rat16_201201_172742']

context_sess = ['Y://fear_conditionning_ptsd\Rat13\Rat13-20201203\Rat13_201203_154214',
               'Y://fear_conditionning_ptsd\Rat14\Rat14-20201204\Rat14_201204_172709',
               'Y://fear_conditionning_ptsd\Rat15\Rat15-20201201\Rat15_201201_155055',
               'Y://fear_conditionning_ptsd\Rat16\Rat16-20201202\Rat16_201202_171552']

fear_sess = ['Y://fear_conditionning_ptsd\Rat13\Rat13-20201203\Rat13_201203_112541',
            'Y://fear_conditionning_ptsd\Rat14\Rat14-20201204\Rat14_201204_113458',
            'Y://fear_conditionning_ptsd\Rat15\Rat15-20201201\Rat15_201201_115713',
            'Y://fear_conditionning_ptsd\Rat16\Rat16-20201202\Rat16_201202_132308']

tone_sess = ['Y://fear_conditionning_ptsd\Rat13\Rat13-20201203\Rat13_201203_164205',
            'Y://fear_conditionning_ptsd\Rat14\Rat14-20201204\Rat14_201204_161859',
            'Y://fear_conditionning_ptsd\Rat16\Rat16-20201202\Rat16_201202_182329']

latetone_sess = ['Y://fear_conditionning_ptsd\Rat13\Rat13-20201211\Rat13_201211_121503',
                'Y://fear_conditionning_ptsd\Rat14\Rat14-20201211\Rat14_201211_122458',
                'Y://fear_conditionning_ptsd\Rat16\Rat16-20201211\Rat16_201211_134118']

latecontext_sess = ['Y://fear_conditionning_ptsd\Rat13\Rat13-20201211\Rat13_201211_113739',
                   'Y://fear_conditionning_ptsd\Rat14\Rat14-20201211\Rat14_201211_114719',
                   'Y://fear_conditionning_ptsd\Rat15\Rat15-20201211\Rat15_201211_141224',
                   'Y://fear_conditionning_ptsd\Rat16\Rat16-20201211\Rat16_201211_142217']


In [4]:
t_start_exp_neutral = [36.98515,25.79435,36.52745,34.03365]

intervals = np.linspace(0,240,13)
starts = intervals[0:-1]
ends = intervals[1:]
intervals = nts.IntervalSet(starts,ends,time_units='s')

ptotal = []
plt.figure()

for fichier,t_s in zip(neutral_sess,t_start_exp_neutral):
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.5)
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-r')
plt.legend(['Rat13', 'Rat14','Rat15','Rat16','Mean'])
plt.title('Pourcent freezing in neutral context before fear conditionning')

Y://fear_conditionning_ptsd\Rat13\Rat13-20201202\Rat13_201202_183404
Y://fear_conditionning_ptsd\Rat14\Rat14-20201203\Rat14_201203_165203
Y://fear_conditionning_ptsd\Rat15\Rat15-20201130\Rat15_201130_164002


ValueError: Length mismatch: Expected axis has 8194 elements, new values have 8195 elements

In [17]:
t_start_exp_fear = [43.6102,61.40995,75.8984,36.58545]

starts = [0,20,40,60,80,100,103,123,138,168,183,213,216]
ends = [20,40,60,80,100,103,123,138,168,183,213,216,240]
intervals = nts.IntervalSet(starts,ends,time_units='s')

ptotal = []
plt.figure()
plt.subplot(3,1,1)

for fichier,t_s in zip(fear_sess,t_start_exp_fear):
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    tone = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[3,:])- t_s * 1_000_000
    shock = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[2,:])- t_s * 1_000_000
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)
bk.plot.intervals(shock,'green',alpha = 0.5)    
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat13', 'Rat14', 'Rat15', 'Rat16','Mean','Shock','Tone'])
plt.title('Pourcent freezing during fear conditionning')

Text(0.5, 1.0, 'Pourcent freezing during fear conditionning')

In [18]:
t_start_exp_tone = [31.5871,32.76115,5.97]

starts = [0,30,60,75,105,120,150,165,195,210]
ends = [30,60,75,105,120,150,165,195,210,240]
intervals = nts.IntervalSet(starts,ends,time_units='s')

ptotal = []
plt.subplot(3,1,2)

for fichier,t_s in zip(tone_sess,t_start_exp_tone):
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    tone = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[3,:])- t_s * 1_000_000
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)
    
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat13', 'Rat14', 'Rat16','Mean','Tone'])
plt.title('Pourcent freezing in tone retrieval')

Text(0.5, 1.0, 'Pourcent freezing in tone retrieval')

In [19]:
t_start_exp_context = [23.49815,31.56275,48.84175,24.49015]

intervals = np.linspace(0,240,13)
starts = intervals[0:-1]
ends = intervals[1:]
intervals = nts.IntervalSet(starts,ends,time_units='s')

ptotal = []
plt.subplot(3,1,3)

for fichier,t_s in zip(context_sess,t_start_exp_context):
    os.chdir(fichier)
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat13', 'Rat14', 'Rat15', 'Rat16','Mean'])
plt.title('Pourcent freezing in context retrieval')

Text(0.5, 1.0, 'Pourcent freezing in context retrieval')

In [56]:
t_start_exp_latetone = [54.28285,58.2436,55.1459]

starts = [0,30,60,75,105,120,150,165,195,210]
ends = [30,60,75,105,120,150,165,195,210,240]
intervals = nts.IntervalSet(starts,ends,time_units='s')

ptotal = []
plt.subplot(2,1,1)

for fichier,t_s in zip(latetone_sess,t_start_exp_latetone):
    os.chdir(fichier)
    if fichier == 'Y://fear_conditionning_ptsd\Rat14\Rat14-20201211\Rat14_201211_122458' :
        pos = DLC_posRatChelou(start = 100_000)
    else : pos = bk.load.DLC_pos()
    tone = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[3,:])- t_s * 1_000_000
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)
    
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat13', 'Rat14','Rat16','Mean','Tone'])
plt.title('Pourcent freezing in late tone retrieval')

Text(0.5, 1.0, 'Pourcent freezing in late tone retrieval')

In [57]:
t_start_exp_latecontext = [51.28165,52.87285,62.79435,52.26585]

intervals = np.linspace(0,240,13)
starts = intervals[0:-1]
ends = intervals[1:]
intervals = nts.IntervalSet(starts,ends,time_units='s')

ptotal = []
plt.subplot(2,1,2)

for fichier,t_s in zip(latecontext_sess,t_start_exp_latecontext):
    os.chdir(fichier)
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat13', 'Rat14', 'Rat15','Rat16', 'Mean'])
plt.title('Pourcent freezing in late context retrieval')

Text(0.5, 1.0, 'Pourcent freezing in late context retrieval')

In [46]:
for subsession in latetone_sess:
    os.chdir(subsession)
    f_tone = GetStartExp(bk.load.digitalin('digitalin.dat')[1,:])
    print(f_tone[f_tone>0])

[54.28285]
[58.2436]
[55.1459]


In [28]:
plt.figure()

intervals = np.linspace(0,240,13)
starts = intervals[0:-1]
ends = intervals[1:]
intervals = nts.IntervalSet(starts,ends,time_units='s')

ptotal=[]
t_s = 51.28165
os.chdir(latecontext_sess[0])

pos = bk.load.DLC_pos()
speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s *1_000_000
pourcents = all_pourcentages(intervals, freeze)

plt.subplot(2,1,1)
bk.plot.intervals(freeze,col = 'orange')
pos.index = pos.index -t_s *1_000_000
plt.axhline(25,color = 'red')
plt.plot(np.mean(speeds.as_units('s'),1))
plt.subplot(2,1,2)
plt.plot(np.mean([starts,ends],0), pourcents, '-o') 
plt.ylim([-10,110])

(-10.0, 110.0)

## Analysis for classical fear conditioning decorrelated

In [4]:
fear_sess = ['Y://classical_fear_conditionning\Rat18\Rat18_201216_115232',
            'Y://classical_fear_conditionning\Rat19\Rat19_201217_115246',
            'Y://classical_fear_conditionning\Rat20\Rat20_201218_115320',
            'Y://classical_fear_conditionning\Rat21\Rat21_201221_113753']

context_sess = ['Y://classical_fear_conditionning\Rat18\Rat18_201216_171056',
               'Y://classical_fear_conditionning\Rat19\Rat19_201217_162502',
               'Y://classical_fear_conditionning\Rat20\Rat20_201218_172829',
               'Y://classical_fear_conditionning\Rat21\Rat21_201221_161044']

tone_sess = ['Y://classical_fear_conditionning\Rat18\Rat18_201216_160525',
            'Y://classical_fear_conditionning\Rat19\Rat19_201217_171115',
            'Y://classical_fear_conditionning\Rat20\Rat20_201218_163147',
            'Y://classical_fear_conditionning\Rat21\Rat21_201221_170813']

intervals = np.linspace(0,600,21)
starts = intervals[0:-1]
ends = intervals[1:]
intervals = nts.IntervalSet(starts,ends,time_units='s')

In [5]:
for dirs,f,fe in os.walk('Y://classical_fear_conditionning/'):
    os.chdir(dirs)
    try: bk.load.DLC_pos(save = True)
    except: print(dirs)
    try: bk.compute.intervals_exp(save = True)
    except: print(dirs) 

Y://classical_fear_conditionning/
Y://classical_fear_conditionning/
Y://classical_fear_conditionning/Rat01-20200630
Y://classical_fear_conditionning/Rat01-20200630
Y://classical_fear_conditionning/Rat02-20200707
Y://classical_fear_conditionning/Rat02-20200707
Y://classical_fear_conditionning/Rat09-20201118
Y://classical_fear_conditionning/Rat09-20201118
Y://classical_fear_conditionning/Rat10-20201118
Y://classical_fear_conditionning/Rat10-20201118
Y://classical_fear_conditionning/Rat11-20201117
Y://classical_fear_conditionning/Rat11-20201117
Y://classical_fear_conditionning/Rat12-20201117
Y://classical_fear_conditionning/Rat12-20201117
Y://classical_fear_conditionning/Rat12-20201117\Rat12_201117_105657
Y://classical_fear_conditionning/Rat12-20201117\Rat12_201117_170955
Y://classical_fear_conditionning/Rat17
Y://classical_fear_conditionning/Rat17
Y://classical_fear_conditionning/Rat17\Rat17_201215_171706
Y://classical_fear_conditionning/Rat17\Rat17_201215_175420
Y://classical_fear_condi

In [5]:
ptotal = []
plt.figure()
plt.subplot(3,1,1)

for fichier in fear_sess:
    os.chdir(fichier)
    exp, shock, tone = bk.compute.intervals_exp()
    t_s = exp.start[0]
    pos = bk.load.DLC_pos()
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s 
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)
tone = tone - t_s
shock = shock - t_s
bk.plot.intervals(shock,'green',alpha = 0.5)    
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat18','Rat19','Rat20','Rat21','Mean','Shock','Tone'])
plt.title('Pourcent freezing during fear conditionning')

FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: 'Y:\\classical_fear_conditionning\\Rat09-20201118\\Rat09_201118_110003'

In [13]:
ptotal = []
plt.subplot(3,1,2)

for fichier in context_sess:
    os.chdir(fichier)
    exp, shock, tone = bk.compute.intervals_exp()
    t_s = exp.start[0]
    pos = bk.load.DLC_pos()
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s 
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)  
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat18','Rat19','Rat20','Rat21'])
plt.title('Pourcent freezing during context recall')

Text(0.5, 1.0, 'Pourcent freezing during context recall')

In [14]:
ptotal = []
plt.subplot(3,1,3)

for fichier in tone_sess:
    os.chdir(fichier)
    exp, shock, tone = bk.compute.intervals_exp()
    t_s = exp.start[0]
    pos = bk.load.DLC_pos()
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s 
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)   
tone = tone - t_s
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat18','Rat19','Rat20','Rat21','Mean','Tone'])
plt.title('Pourcent freezing during tone recall')

Text(0.5, 1.0, 'Pourcent freezing during tone recall')

# Exp 3

In [135]:
fear_sess = ['Z://classical_fear_conditionning\Rat23\Rat23-20210128_210128_103618',
            'Z://classical_fear_conditionning\Rat24\Rat24_20210129_210129_113321']

context_sess = ['Z://classical_fear_conditionning\Rat23\Rat23-20210128_210128_143126',
               'Z://classical_fear_conditionning\Rat24\Rat24_20210129_210129_152259']

tone_sess = ['Z://classical_fear_conditionning\Rat23',
    'Z://classical_fear_conditionning\Rat24\Rat24-20210129_210129_163341']

intervals = np.linspace(0,600,21)
starts = intervals[0:-1]
ends = intervals[1:]
intervals = nts.IntervalSet(starts,ends,time_units='s')

In [134]:
for dirs,f,fe in os.walk('Z://classical_fear_conditionning/'):
    os.chdir(dirs)
    try: bk.load.DLC_pos(save = True)
    except: print(dirs)
    try: bk.compute.intervals_exp(save = True)
    except: print(dirs) 

Z://classical_fear_conditionning/
Z://classical_fear_conditionning/
Z://classical_fear_conditionning/Rat01-20200630
Z://classical_fear_conditionning/Rat01-20200630
Z://classical_fear_conditionning/Rat02-20200707
Z://classical_fear_conditionning/Rat02-20200707
Z://classical_fear_conditionning/Rat09-20201118
Z://classical_fear_conditionning/Rat09-20201118
Z://classical_fear_conditionning/Rat10-20201118
Z://classical_fear_conditionning/Rat10-20201118
Z://classical_fear_conditionning/Rat11-20201117
Z://classical_fear_conditionning/Rat11-20201117
Z://classical_fear_conditionning/Rat12-20201117
Z://classical_fear_conditionning/Rat12-20201117
Z://classical_fear_conditionning/Rat12-20201117\Rat12_201117_105657
Z://classical_fear_conditionning/Rat12-20201117\Rat12_201117_170955
Z://classical_fear_conditionning/Rat17
Z://classical_fear_conditionning/Rat17
Z://classical_fear_conditionning/Rat17\Rat17_201215_171706
Z://classical_fear_conditionning/Rat17\Rat17_201215_175420
Z://classical_fear_condi

In [138]:
ptotal = []
plt.figure()
plt.subplot(3,1,1)

for fichier in fear_sess:
    os.chdir(fichier)
    exp, shock, tone = bk.compute.intervals_exp()
    t_s = exp.start[0]
    pos = bk.load.DLC_pos()
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail', 'b_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s 
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)
tone = tone - t_s
shock = shock - t_s
bk.plot.intervals(shock,'green',alpha = 0.5)    
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat23', 'Rat24','Mean','Shock','Tone'])
plt.title('Pourcent freezing during fear conditionning')

ValueError: Length mismatch: Expected axis has 19202 elements, new values have 19330 elements

In [139]:
ptotal = []
plt.subplot(3,1,2)

for fichier in context_sess:
    os.chdir(fichier)
    exp, shock, tone = bk.compute.intervals_exp()
    t_s = exp.start[0]
    pos = bk.load.DLC_pos()
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail', 'b_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s 
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)  
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat23', 'Rat24', 'Mean'])
plt.title('Pourcent freezing during context recall')

ValueError: Length mismatch: Expected axis has 19159 elements, new values have 19226 elements

In [ ]:
ptotal = []
plt.subplot(3,1,3)

for fichier in tone_sess:
    os.chdir(fichier)
    exp, shock, tone = bk.compute.intervals_exp()
    t_s = exp.start[0]
    pos = bk.load.DLC_pos()
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail', 'b_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s 
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)   
tone = tone - t_s
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat23', 'Rat24','Mean','Tone'])
plt.title('Pourcent freezing during tone recall')

## Old stuff

In [43]:
ptotal = []
plt.figure()
plt.subplot(3,1,1)

for fichier in fear_sess:
    os.chdir(fichier)
    start_sess = GetStartExp(bk.load.digitalin('digitalin.dat')[1,:])
    t_s = start_sess[start_sess>0][0]
    pos = bk.load.DLC_pos()
    speeds = bk.compute.speed(pos,2, columns_to_drop=['b_tail','e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)
tone = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[3,:])- t_s * 1_000_000
shock = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[2,:])- t_s * 1_000_000
bk.plot.intervals(shock,'green',alpha = 0.5)    
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat18','Rat19','Rat20','Mean','Shock','Tone'])
plt.title('Pourcent freezing during fear conditionning')

Text(0.5, 1.0, 'Pourcent freezing during fear conditionning')

In [65]:
ptotal = []
plt.subplot(3,1,2)

for fichier in context_sess:
    os.chdir(fichier)
    start_sess = GetStartExp(bk.load.digitalin('digitalin.dat')[1,:])
    t_s = start_sess[start_sess>0][0]
    pos = bk.load.DLC_pos()
    speeds = bk.compute.speed(pos,2, columns_to_drop=['b_tail','e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)  
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat18', 'Rat19','Rat20','Mean'])
plt.title('Pourcent freezing during context recall')

Text(0.5, 1.0, 'Pourcent freezing during context recall')

In [66]:
ptotal = []
plt.subplot(3,1,3)

for fichier in tone_sess:
    os.chdir(fichier)
    start_sess = GetStartExp(bk.load.digitalin('digitalin.dat')[1,:])
    t_s = start_sess[start_sess>0][0]
    pos = bk.load.DLC_pos()
    speeds = bk.compute.speed(pos,2, columns_to_drop=['b_tail','e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)   
tone = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[3,:])- t_s * 1_000_000
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat18', 'Rat19','Rat20','Mean','Tone'])
plt.title('Pourcent freezing during tone recall')

Text(0.5, 1.0, 'Pourcent freezing during tone recall')

In [132]:
for dirs,f,fe in os.walk('Z://classical_fear_conditionning/'):
    os.chdir(dirs)
    if os.path.exists('digitalin.dat'):
        exp, shock, tone = bk.compute.intervals_exp()
        t_s = exp.start[0]
        pos = bk.load.DLC_pos()
        speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail','b_tail'])
        freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s

        freeze_in_tone = freeze.intersect(tone).as_units('s')
        freeze_in_shock = freeze.intersect(shock).as_units('s')
        freeze_in_nothing = freeze.set_diff(tone.union(shock)).as_units('s')

        list_tone = ['tone' for i in range (len(freeze_in_tone))]
        list_shock = ['shock' for i in range (len(freeze_in_shock))]
        list_nothing = ['nothing' for i in range (len(freeze_in_nothing))]

        freeze_in_tone = pd.DataFrame(freeze_in_tone.set_axis(list_tone))
        freeze_in_shock = pd.DataFrame(freeze_in_shock.set_axis(list_shock))
        freeze_in_nothing = pd.DataFrame(freeze_in_nothing.set_axis(list_nothing))

        for data in [freeze_in_nothing, freeze_in_shock, freeze_in_tone]:
            data.to_csv('raw_freezing_intervals.csv', header=False, mode='a')

KeyError: 0

In [57]:
for dirs,f,fe in os.walk('Z://classical_fear_conditionning/'):
    os.chdir(dirs)
    if os.path.exists('digitalin.dat'):
        print(dirs)
        exp, shock, tone = bk.compute.intervals_exp()
        try: pos = bk.load.DLC_pos()
        except: continue
        speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
        plt.figure()
        plt.plot(np.mean(speeds.as_units('s'),1), color='r')
        bk.plot.intervals(tone, col='green')
        bk.plot.intervals(shock, col='red')
        plt.savefig('speed.png')

Z://classical_fear_conditionning/Rat01-20200630\rat01_200630_114732
Z://classical_fear_conditionning/Rat01-20200630\rat01_200630_171900
Z://classical_fear_conditionning/Rat01-20200630\rat01_200630_183149
Z://classical_fear_conditionning/Rat02-20200707\rat02_200707_114611
Z://classical_fear_conditionning/Rat02-20200707\rat02_200707_172929
Z://classical_fear_conditionning/Rat02-20200707\rat02_200707_183427
Z://classical_fear_conditionning/Rat09-20201118\Rat09_201118_110003
Z://classical_fear_conditionning/Rat09-20201118\Rat09_201118_170529
Z://classical_fear_conditionning/Rat09-20201118\Rat09_201118_180813
Z://classical_fear_conditionning/Rat10-20201118\Rat10_201118_111531
Z://classical_fear_conditionning/Rat10-20201118\Rat10_201118_172028
Z://classical_fear_conditionning/Rat10-20201118\Rat10_201118_182234
Z://classical_fear_conditionning/Rat11-20201117\Rat11_201117_112130
Z://classical_fear_conditionning/Rat11-20201117\Rat11_201117_172705
Z://classical_fear_conditionning/Rat11-20201117\

Y:\Python\Maelle\Amy-Hpc-sleep-dynamics-python\bk\compute.py:161: RuntimeWarning: overflow encountered in square
  v = np.sqrt(x_speed**2 + y_speed**2)
<ipython-input-57-1f23888b18ae>:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


Z://classical_fear_conditionning/Rat17\Rat17_201215_162301
Z://classical_fear_conditionning/Rat17\Rat17_201215_171706
Z://classical_fear_conditionning/Rat17\Rat17_201215_175420
Z://classical_fear_conditionning/Rat17\Rat17_201215_175705
Z://classical_fear_conditionning/Rat17\Rat17_201215_180057
Z://classical_fear_conditionning/Rat18\Rat18_201216_115232
Z://classical_fear_conditionning/Rat18\Rat18_201216_160525
Z://classical_fear_conditionning/Rat18\Rat18_201216_171056
Z://classical_fear_conditionning/Rat18\Rat18_201222_110919
Z://classical_fear_conditionning/Rat19\Rat19_201217_115246
Z://classical_fear_conditionning/Rat19\Rat19_201217_162502
Z://classical_fear_conditionning/Rat19\Rat19_201217_171115
Z://classical_fear_conditionning/Rat19\Rat19_201222_112653
Z://classical_fear_conditionning/Rat20\Rat20_201218_115320
Z://classical_fear_conditionning/Rat20\Rat20_201218_163147
Z://classical_fear_conditionning/Rat20\Rat20_201218_172829
Z://classical_fear_conditionning/Rat20\Rat20_201222_1143